# Treinamento & Validação
Esse **Jupyter Notebook** tem como objetivo treinar modelos de *Machine Learning* com uma ou mais *features* e verificar quão bem esses modelos estão aprendendo com base na métrica de validação - [Erro Médio Absoluto](https://en.wikipedia.org/wiki/Mean_absolute_error).

---

# Classes "Training" e "Validation"
Como um dos requisitos da **GRIA** para o desafio era que os códigos fossem reaproveitados e documentados vamos criar a classe **Training** que vai ser Responsável pelo processo de treinar nossos dados em vários modelos diferentes e metrifica-los.

In [1]:
class Training:
  pass

---

# 01 - Preparando os dados & Ambiente
Nessa etapa vamos preparar os dados e o ambiente (jupyter notebook) para **treinar** e **validar** nossos modelos.

---

## 01.1 - Baixando as Bibliotecas necessárias
Inicialmente vamos baixar as bibliotecas necessárias para nossa análise (Eu já tenho todas baixadas no meu ambiente virtual mas você pode remover o comentário e baixar para sua máquina local ou Ambiente Virtual).

In [2]:
#!pip install --upgrade -r ../requirements.txt --user

---

## 01.2 - Importando o módulo "Preprocessing"
Nós também vamos utilizar o módulo **"Preprocessing"** que foi criado na etapa de *Pré-Processamento*. Lembrem, que nós também temos que *Pré-Processar* os dados de *teste* como foi feito nos dados de treino e vamos utilizar os métodos de extrair e pegar os conjuntos de dados 

In [3]:
%run "../src/preprocessing.py"

In [4]:
preprocessing = Preprocessing()

---

## 01.3 - Extraindo os conjuntos de dados de treino e teste
Os conjuntos de dados de **treino** e **teste** vão ser utilizados quase que sempre para treinar e validar nossos modelos. Então, vamos deixar ambos já extraídos para trabalhos futuros.

In [8]:
# Extract training set.
preprocessing.extract_7z_data("../datasets/Train_rev1.7z")

File extracted!


In [9]:
# Extract testing set.
preprocessing.extract_7z_data("../datasets/Test_rev1.7z")

File extracted!


---

## 01.4 - Pegando o conjunto de dados de teste

Agora vamos pegar o conjunto de dados de teste disponibilizado pelo **Adzuna**.

**NOTE:**  
Vale lembrar que nesse conjunto de dados não tem as colunas (feature) **"SalaryRaw"** e **"SalaryNormalized"**.

In [6]:
df_testing = preprocessing.get_testing_data()

Testing data ready!


In [7]:
df_testing.info()
df_testing.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 122463 entries, 0 to 122462
Data columns (total 10 columns):
 #   Column              Non-Null Count   Dtype 
---  ------              --------------   ----- 
 0   Id                  122463 non-null  int64 
 1   Title               122463 non-null  object
 2   FullDescription     122463 non-null  object
 3   LocationRaw         122463 non-null  object
 4   LocationNormalized  122463 non-null  object
 5   ContractType        33013 non-null   object
 6   ContractTime        90702 non-null   object
 7   Company             106202 non-null  object
 8   Category            122463 non-null  object
 9   SourceName          122463 non-null  object
dtypes: int64(1), object(9)
memory usage: 9.3+ MB


,Id,Title,FullDescription,LocationRaw,LocationNormalized,ContractType,ContractTime,Company,Category,SourceName
0,11888454,Business Development Manager,The Company: Our client is a national training...,"Tyne Wear, North East",Newcastle Upon Tyne,NaN,permanent,Asset Appointments,Teaching Jobs,cv-library.co.uk
1,11988350,Internal Account Manager,The Company: Founded in **** our client is a U...,"Tyne and Wear, North East",Newcastle Upon Tyne,NaN,permanent,Asset Appointments,Consultancy Jobs,cv-library.co.uk
2,12612558,Engineering Systems Analysts,Engineering Systems Analysts Surrey ****K Loca...,"Surrey, South East, South East",Surrey,NaN,permanent,Gregory Martin International,Engineering Jobs,cv-library.co.uk
3,12613014,CIS Systems Engineering Consultant,CIS Systems Engineering Consultant Bristol So...,"Bristol, South West, South West",Bristol,NaN,permanent,Gregory Martin International,Engineering Jobs,cv-library.co.uk
4,22454872,CNC Miller / Programmer Fanac,"CNC Miller / Programmer Fanac Fleet, Hampshire...","Fleet, Hampshire",Fleet,NaN,permanent,Gregory Martin International,Manufacturing Jobs,cv-library.co.uk


---

# 03 - Treinando & Validando os Loads
> Na parte de **Treinamento & Validação** nós vamos utilizar as colunas (features) já Pré-Processadas em cada **Load** para treinar vários modelos de Regressão e tentar encontrar o que nós dar o melhor resultado de acordo com os dados passados.

---

## 03.1 - Treinando & Validando o Load-v1
Bem, como nós sabemos no **load-v1** foi passado para a etapa de **treinamento & validação** a coluna (feature) **"Title"**. Ou seja, nós vamos ter as seguintes variáveis (features) para os nossos modelos:

 - **Variáveis Independente:**
   - Title *(com CountVectorizer):*
     - stop_words="english"
     - max_df=0.60 (Ignores terms that appear in MORE than 60% of documents)
     - min_df=0.05 (Ignores terms that appear in LESS than 5% of documents)
 - **Variáveis Dependente:**
   - SalaryNormalized (normalizada pelo a Adzuna)

**NOTE:**  
Esse vai ser o nosso **baseline model**.

**Pegando a variável dependente (target):**

In [11]:
df_training = preprocessing.get_training_data()

AttributeError: 'Preprocessing' object has no attribute 'get_training_data'

In [15]:
import py7zr

with py7zr.SevenZipFile("../datasets/Train_rev1.7z", mode='r') as archive:
  archive.extractall(path="/tmp") # For Linux users.

In [16]:
import pandas as pd

full_df = pd.read_csv("/tmp/Train_rev1.csv")
df_SalaryNormalized = full_df[["SalaryNormalized"]]
df_SalaryNormalized.info()
df_SalaryNormalized.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 244768 entries, 0 to 244767
Data columns (total 1 columns):
 #   Column            Non-Null Count   Dtype
---  ------            --------------   -----
 0   SalaryNormalized  244768 non-null  int64
dtypes: int64(1)
memory usage: 1.9 MB


,SalaryNormalized
0,25000
1,30000
2,30000
3,27500
4,25000


### Pegando a(s) variável(s) Independente(s):

In [17]:
import scipy.sparse
df_title_vectorized = scipy.sparse.load_npz('df_title_vectorized.npz')

In [18]:
df_title_vectorized

<244768x14917 sparse matrix of type '<class 'numpy.int64'>'
	with 923171 stored elements in Compressed Sparse Row format>

### Separando as variáveis em "x" e "y":

In [19]:
x = df_title_vectorized
y = df_SalaryNormalized

### Dividindo os dados em dados de treino e dados de validação:

In [20]:
from sklearn.model_selection import train_test_split
x_train, x_valid, y_train, y_valid = train_test_split(x, y, test_size=0.3, random_state=42)

### Treinando modelos de Regressão:

#### Regressão Linear:

In [46]:
# Linear Regression.
from sklearn.linear_model import LinearRegression
lrModel = LinearRegression() # Instance.
lrModel.fit(x_train, y_train) # Training.

salary_predicted_ln = lrModel.predict(x_valid)
print('Predictions:\n ', salary_predicted_ln)

from sklearn.metrics import mean_absolute_error
mae_ln = mean_absolute_error(y_valid, salary_predicted_ln)
print('MAE for Linear Regression Model:', round(mae_ln))

Predictions:
  [[36329.20127848]
 [24914.92159844]
 [30252.21978656]
 ...
 [21815.84460272]
 [30797.87181137]
 [34741.34393378]]
MAE for Linear Regression Model: 8658


#### Ridge Regression (L2):

In [48]:
from sklearn.linear_model import Ridge
ridgeModel = Ridge() # Instance.
ridgeModel.fit(x_train, y_train) # Training.

salary_predicted_ridge = ridgeModel.predict(x_valid)
print('Predictions:\n ', salary_predicted_ridge)

from sklearn.metrics import mean_absolute_error
mae_ridge = mean_absolute_error(y_valid, salary_predicted_ridge)
print('MAE for Ridge Model:', round(mae_ridge))

Predictions:
  [[36282.66675248]
 [17353.9188599 ]
 [30256.92827149]
 ...
 [23060.25942236]
 [28012.35552722]
 [34769.28500259]]
MAE for Ridge Model: 8617


#### Lasso Regression (L1):

In [49]:
from sklearn.linear_model import Lasso
lassoModel = Lasso() # Instance.
lassoModel.fit(x_train, y_train) # Training.

salary_predicted_lasso = lassoModel.predict(x_valid)
print('Predictions:\n ', salary_predicted_lasso)

from sklearn.metrics import mean_absolute_error
mae_lasso = mean_absolute_error(y_valid, salary_predicted_lasso)
print('MAE for Lasso Model:', round(mae_lasso))

Predictions:
  [36249.43889789 20163.56243617 30465.88439844 ... 32526.98079274
 25040.10621769 39581.57774515]
MAE for Lasso Model: 8834


#### Elastic Net:

In [50]:
from sklearn.linear_model import ElasticNet
elasticNetModel = ElasticNet() # Instance.
elasticNetModel.fit(x_train, y_train) # Training.

salary_predicted_elasticNet = elasticNetModel.predict(x_valid)
print('Predictions:\n ', salary_predicted_elasticNet)

from sklearn.metrics import mean_absolute_error
mae_elasticNet = mean_absolute_error(y_valid, salary_predicted_elasticNet)
print('MAE for Elastic Net Model:', round(mae_elasticNet))

Predictions:
  [35811.18015834 32896.16153646 35328.01934386 ... 33810.29474899
 33205.52933158 35922.86842709]
MAE for Elastic Net Model: 12888


#### Random Forest Regressor:

In [40]:
from sklearn.ensemble import RandomForestRegressor
import numpy as np

RandomFRModel = RandomForestRegressor(n_jobs=-1) # Instance.
RandomFRModel.fit(x_train, np.ravel(y_train)) # Training.

salary_predicted_RandomFR = RandomFRModel.predict(x_valid)
print('Predictions:\n ', salary_predicted_RandomFR)

from sklearn.metrics import mean_absolute_error
mae_RandomFR = mean_absolute_error(y_valid, salary_predicted_RandomFR.ravel())
print('MAE for RandomForestRegressor Model:', round(mae_RandomFR))

Predictions:
  [31102.85583861 30417.05190476 27328.22505411 ... 19981.93197619
 27823.39497785 23481.06557147]
MAE for RandomForestRegressor Model:  6703.093421570222


---

# Resumos

 - **No *Load-v1* nós tinhamos as seguintes situações:**
   - Variáveis (features):
     - Independentes:
       - Title *(com CountVectorizer):*
         - stop_words="english"
         - max_df=0.60 (Ignores terms that appear in MORE than 60% of documents)
         - min_df=0.05 (Ignores terms that appear in LESS than 5% of documents)
     - Dependente:
       - SalaryNormalized (normalizada pelo a Adzuna)
   - *Como Métrica de Avaliação (MAE) tivemos os seguintes resultados (ordenados do menor para o maior):*
     - MAE for RandomForestRegressor Model: 6703
     - MAE for Ridge Model: 8616
     - MAE for Linear Regression Model: 8658
     - MAE for Lasso Model: 8834
     - MAE for Elastic Net Model: 12888

**Rodrigo Leite -** *drigols*